# Model

In [1]:
# from google.colab import userdata
# tok = userdata.get('git_token')

In [2]:
# !pip install torchmetrics -Uqq
from torchmetrics.classification import BinaryAccuracy

/home/dai/anaconda3/envs/pytorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [126]:
import pandas as pd
import torch
from torch import nn
from torch.nn import functional as F
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader,Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler
import numpy as np


In [46]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [127]:
import torch.nn as nn

class ClsModel(nn.Module):
    def __init__(self, input_size, hidden_size, out_size, dropout_rate=0.3):
        super().__init__()
        self.model = nn.Sequential(
          nn.Linear(in_features=input_size, out_features=50, dtype=torch.float64),
          nn.ReLU(),
          nn.Linear(in_features=50, out_features=50, dtype=torch.float64),
          nn.ReLU(),
          nn.Linear(in_features=50, out_features=50, dtype=torch.float64),
          nn.ReLU(),
          nn.Dropout(0.5),
          nn.Linear(in_features=50, out_features=20, dtype=torch.float64),
          nn.ReLU(),
          nn.Dropout(0.3),
          nn.Linear(in_features=20, out_features=20, dtype=torch.float64),
          nn.ReLU(),
          nn.Linear(in_features=20, out_features=hidden_size, dtype=torch.float64),
          nn.ReLU(),
          nn.Linear(in_features=hidden_size, out_features=hidden_size, dtype=torch.float64),
          nn.ReLU(),
          nn.Linear(in_features=hidden_size, out_features=hidden_size, dtype=torch.float64),
          nn.ReLU(),
          nn.Linear(in_features=hidden_size, out_features=hidden_size, dtype=torch.float64),
          nn.ReLU(),
          nn.Linear(in_features=hidden_size, out_features=hidden_size, dtype=torch.float64),
          nn.ReLU(),
          nn.Dropout(0.3),
          nn.Linear(in_features=hidden_size, out_features=hidden_size, dtype=torch.float64),
          nn.ReLU(),
          nn.Dropout(0.5),
          nn.Linear(in_features=hidden_size, out_features=out_size, dtype=torch.float64),
          nn.Sigmoid()
      )

    def forward(self, data):
        return self.model(data)


In [57]:
from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# df = scaler.fit_transform(df)

In [112]:
def prepare_train(df):
    categorical_cols = ["hearing(left)", "hearing(right)","dental caries"]
    numerical_cols = [col for col in df.columns if col not in categorical_cols]

    df["bmi"] = df["weight(kg)"] / (df["height(cm)"]/100)**2
    df.drop(["height(cm)", "weight(kg)"], axis=1, inplace=True)

    y = df["smoking"]

    numerical_cols.remove("height(cm)")
    numerical_cols.remove("weight(kg)")
    numerical_cols.remove("smoking")
    numerical_cols.append("bmi")
    df.drop(["smoking"],axis=1,inplace=True)

    scaler = StandardScaler()
    df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

    return df.to_numpy(), y, scaler

df=pd.read_csv("train.csv", index_col='id')
df,y,scl = prepare_train(df)

In [113]:
def prepare_test(df, scaler):
    categorical_cols = ["hearing(left)", "hearing(right)", "dental caries"]
    numerical_cols = [col for col in df.columns if col not in categorical_cols]
    df["bmi"] = df["weight(kg)"] / (df["height(cm)"]/100)**2
    df.drop(["height(cm)", "weight(kg)"], axis=1, inplace=True)
    numerical_cols.remove("height(cm)")
    numerical_cols.remove("weight(kg)")
    numerical_cols.append("bmi")
    df[numerical_cols] = scaler.transform(df[numerical_cols])
    return df

test = pd.read_csv("test.csv", index_col='id')
test = prepare_test(test, scl)

In [114]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x_sample = self.X[idx]
        y_sample = self.y.iloc[idx]
        return x_sample.astype("float64"), y_sample.astype("float64")

In [115]:
# def get_data_loaders(X,y):
#     X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,shuffle=True,stratify=y)
#     train_dataset = CustomDataset(X,y)
#     val_dataset = CustomDataset(X_test,y_test)
#     train_loader = DataLoader(train_dataset,batch_size=32,num_workers=os.cpu_count(),pin_memory=True)
#     val_loader = DataLoader(val_dataset,batch_size=32,num_workers=os.cpu_count(),pin_memory=True)
#     return train_loader, val_loader
# train_loader, val_loader = get_data_loaders(df,y)

In [119]:
from sklearn.model_selection import StratifiedKFold

def get_cv_data_loaders(X, y, n_splits=5):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    for train_index, val_index in skf.split(X, y):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        train_dataset = CustomDataset(X_train, y_train)
        val_dataset = CustomDataset(X_val, y_val)

        train_loader = DataLoader(train_dataset, batch_size=32, num_workers=os.cpu_count(), pin_memory=True)
        val_loader = DataLoader(val_dataset, batch_size=32, num_workers=os.cpu_count(), pin_memory=True)

        yield train_loader, val_loader

In [77]:
def save_checkpoint(model, optimizer, epoch, loss, filename):
    path = filename + '.pth'
    torch.save({
      'epoch': epoch,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'loss': loss,
  }, path)

def load_checkpoint(model, optimizer, filename):
    path = filename + '.pth'
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    model.eval()
    return model, optimizer, epoch, loss

In [78]:
torch.manual_seed(42)
model = ClsModel(22,10,1)
model.to(device)
# model.compile()
loss_fn = nn.BCELoss()
adam = torch.optim.Adam(model.parameters(),lr=0.001)
scheduler = ReduceLROnPlateau(adam, 'min', verbose=True)

In [79]:
def training_loop():
    for i in range(epochs):
    #Training
        model.train()
        accumulated_loss = 0
        print(f"Epoch: {i}")
        for idx,batch in enumerate(train_loader):
            msg = f"batch: {idx}"
            print('\r' + msg, end='', flush=True)
            X,y = batch
            X = X.to(device)
            y = y.to(device)
            preds = model(X)
            loss = loss_fn(preds,y.reshape(-1,1))
            adam.zero_grad()
            loss.backward()
            adam.step()
            accumulated_loss += loss.item()
        print()
        _loss = accumulated_loss/len(train_loader)
        scheduler.step(_loss)
        print(accumulated_loss/len(train_loader))
        save_checkpoint(model, adam, i, accumulated_loss/len(train_loader), 'model')

training_loop()

Epoch: 0


KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/dai/anaconda3/envs/pytorch/lib/python3.11/site-packages/pandas/core/indexes/base.py", line 3802, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "pandas/_libs/index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 165, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 5745, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 5753, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 0

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/dai/anaconda3/envs/pytorch/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
        

In [ ]:
# os.cpu_count()

In [ ]:
# df=pd.read_csv("/content/train.csv")
# y=df["smoking"]

# df=df.drop(["id", "smoking"],axis=1)
# X_train,X_test,y_train,y_test = train_test_split(df,y,random_state=42,shuffle=True,stratify=y)

In [ ]:
# test_X = torch.Tensor(X_test.to_numpy()).to(torch.float64)
# test_y = torch.Tensor(y_test.to_numpy()).to(torch.float64)

In [ ]:
# metrics = BinaryAccuracy().to(device)

In [ ]:
# with torch.inference_mode():
#     preds = model(test_X.to(device))
#     acc = metrics(preds,test_y.reshape(-1,1).to(device))
#     print(acc)

In [23]:
# with torch.inference_mode():
#     preds = model(test_X.to(device))
#     acc = metrics(preds,test_y.reshape(-1,1).to(device))
#     print(acc)

In [24]:
with torch.inference_mode():
    preds = model(test.to(device))
    print(preds)

tensor([[5.6437e-01],
        [3.6937e-01],
        [1.4972e-01],
        ...,
        [4.2727e-01],
        [6.2649e-02],
        [1.3695e-05]], dtype=torch.float64)


In [25]:
submission = pd.read_csv("sample_submission.csv", index_col="id")
submission['smoking'] = preds.cpu()
submission.to_csv("submission.csv")